In [ ]:
# Cell 0 — install timm and mount drive (run if needed)
!pip install timm --quiet

from google.colab import drive
drive.mount('/content/drive')


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# 1.**Import Most Needed Libraries**

In [1]:
# Cell 1 — imports and constants
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from PIL import Image
import timm

# Path to your dataset root in Drive (adjust if different)
DATASET_ROOT = r"A:\PyTorch\Project\140k_Real_Fake_Faces\real_vs_fake\real-vs-fake"  # <-- edit if needed

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

a:\PyTorch\Environment\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda


# 2.**DeepfakeDataset class**

In [2]:
# Cell 2 — DeepfakeDataset class
class DeepfakeDataset(Dataset):
    def __init__(self, root, split="train", transform=None):
        self.root = os.path.join(root, split)
        self.transform = transform
        self.samples = []
        for label_name, label_value in [("real", 0), ("fake", 1)]:
            folder = os.path.join(self.root, label_name)
            if not os.path.isdir(folder):
                continue
            for filename in os.listdir(folder):
                fpath = os.path.join(folder, filename)
                if os.path.isfile(fpath):
                    self.samples.append((fpath, label_value))
    def __len__(self):
        return len(self.samples)
    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor([label], dtype=torch.float32)

# 3.**Transforms for Xception**

In [3]:
# Cell 3 — transforms for Xception
train_transform = T.Compose([
    T.Resize((342, 342)),
    T.RandomResizedCrop(299, scale=(0.65, 1.0)),   # crop and resize to 299x299
    T.RandomHorizontalFlip(p=0.5),
    T.RandomApply([T.ColorJitter(0.4,0.4,0.2,0.1)], p=0.7),   # brightness, contrast, saturation, hue
    T.RandomApply([T.GaussianBlur(kernel_size=5)], p=0.3),    # blur with a 5x5 kernel matrix
    T.RandomApply([T.RandomPosterize(bits=4)], p=0.2),        # reduce to 4 bits for each channel
    T.RandomGrayscale(p=0.02),                    # convert to grayscale with 2% probability
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3),             # normalize to [-1, 1]
])

val_transform = T.Compose([
    T.Resize((299, 299)),
    T.ToTensor(),
    T.Normalize([0.5]*3, [0.5]*3),
])

# 4.**Create Datasets & Dataloaders**

In [4]:
# Cell 4 — create datasets & dataloaders
batch_size = 32   # reduce to 16 if OOM
num_workers = 4

train_ds = DeepfakeDataset(DATASET_ROOT, "train", train_transform)
val_ds   = DeepfakeDataset(DATASET_ROOT, "val", val_transform)
test_ds  = DeepfakeDataset(DATASET_ROOT, "test", val_transform)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

print("Dataset sizes — train:", len(train_ds), "val:", len(val_ds), "test:", len(test_ds))

Dataset sizes — train: 100000 val: 20000 test: 20000


# 5.**Xception model with dropout head**

In [ ]:
# Cell 5 — Xception model with dropout head
def get_xception_model(dropout=0.3):
    base = timm.create_model("xception", pretrained=True)
    # Detect which classification layer exists and replace it
    if hasattr(base, "fc"):
        in_features = base.fc.in_features
    elif hasattr(base, "classifier"):
        in_features = base.classifier.in_features
    else:
        raise RuntimeError("Unknown xception head name")
    base.fc = nn.Sequential(
        nn.Dropout(p=dropout),
        nn.Linear(in_features, 1)
    )
    return base

model = get_xception_model().to(device)
print(model)

# 6.**Loss, Optimizer, Scheduler, Scaler**

In [ ]:
# Cell 6 — loss, optimizer, scheduler, scaler
criterion = nn.BCEWithLogitsLoss()

# param groups
head_params = [p for n,p in model.named_parameters() if "fc" in n]
backbone_params = [p for n,p in model.named_parameters() if "fc" not in n]

optimizer = torch.optim.AdamW([
    {"params": backbone_params, "lr": 1e-5, "weight_decay": 1e-4},
    {"params": head_params,     "lr": 1e-4, "weight_decay": 1e-3},
])

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", factor=0.5, patience=2, verbose=True, min_lr=1e-7
)

scaler = torch.cuda.amp.GradScaler()   # for mixed precision training
print("Optimizer and scheduler created.")

# 7.**Checkpoint Helpers**

In [7]:
# Cell 7 — checkpoint helpers
CHECKPOINT_DIR = r"A:\PyTorch"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

def save_checkpoint(epoch, model, optimizer, scaler, scheduler, best_val_acc, filename):
    checkpoint = {
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "scaler_state": scaler.state_dict() if scaler is not None else None,
        "scheduler_state": scheduler.state_dict(),
        "best_val_acc": best_val_acc,
        "model_name": model.__class__.__name__,
        "timm_name": getattr(model, "default_cfg", {}).get("architecture", None)
    }
    torch.save(checkpoint, filename)
    print(f"💾 Saved: {filename}")

def load_checkpoint(path, model, optimizer, scaler, scheduler):
    print(f"🔄 Loading checkpoint: {path}")
    checkpoint = torch.load(path, map_location=device)
    saved_name = checkpoint.get("model_name", None)
    current_name = model.__class__.__name__
    if saved_name is not None and saved_name != current_name:
        print(f"⚠️ WARNING: Checkpoint was trained on: {saved_name}")
        print(f"⚠️ Current model is: {current_name}")
        print("⚠️ Loading anyway — ensure architecture compatibility.")
    
    model.load_state_dict(checkpoint["model_state"], strict=True)
    optimizer.load_state_dict(checkpoint["optimizer_state"])
    if checkpoint["scaler_state"] is not None and scaler is not None:
        scaler.load_state_dict(checkpoint["scaler_state"])
    scheduler.load_state_dict(checkpoint["scheduler_state"])
    start_epoch = checkpoint["epoch"] + 1
    best_val_acc = checkpoint["best_val_acc"]
    print(f"✔ Resumed from epoch {start_epoch}, best_val_acc={best_val_acc:.2f}%")
    return start_epoch, best_val_acc

# 8.**Train & Validate**

In [8]:
# Cell 8 — train & validate
from tqdm import tqdm

def train_one_epoch(model, train_loader, criterion, optimizer, scaler, device):
    model.train()        # set to training mode
    running_loss = 0.0   # cumulative loss (batch)
    correct = 0          # correct predictions
    total = 0            # total samples
    loop = tqdm(train_loader, leave=False)  # progress bar
    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()   # reset gradients
        # AMP new syntax
        with torch.amp.autocast(device_type="cuda"):
            outputs = model(imgs)
            loss = criterion(outputs, labels)  # compute loss
        # Backprop with scaling & clipping
        scaler.scale(loss).backward()   # scaled backward pass
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   # gradient clipping
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item() * imgs.size(0)
        preds = (torch.sigmoid(outputs) >= 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        loop.set_description(f"loss: {loss.item():.4f}")
    return running_loss / total, correct / total * 100

def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)
            preds = (torch.sigmoid(outputs) >= 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total * 100

# 9.**Main Training Loop**

In [ ]:
# Cell 9 — main training loop
resume = False
checkpoint_path = f"{CHECKPOINT_DIR}/checkpoint_epoch_6.pth"  # edit if you have a checkpoint

if resume:
    start_epoch, best_val_acc = load_checkpoint(checkpoint_path, model, optimizer, scaler, scheduler)
else:
    start_epoch = 0
    best_val_acc = 0.0

epochs = 8
patience = 3
no_improve = 0

for epoch in range(start_epoch, epochs):
    print(f"\n🔵 Epoch {epoch+1}/{epochs}")
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, scaler, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.2f}%")
    print(f" Val  Loss: {val_loss:.4f} | Acc: {val_acc:.2f}%")
    scheduler.step(val_acc)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        no_improve = 0
        save_checkpoint(epoch, model, optimizer, scaler, scheduler, best_val_acc, f"{CHECKPOINT_DIR}/best_model.pth")
        print("🏆 Best model updated!")
    else:
        no_improve += 1
    if no_improve >= patience:
        print("⛔ Early stopping triggered!")
        break
    save_checkpoint(epoch, model, optimizer, scaler, scheduler, best_val_acc, f"{CHECKPOINT_DIR}/checkpoint_epoch_{epoch+1}.pth")

# 10.**Evaluation & Single-Image Prediction**

In [ ]:
# Cell 10 — evaluation & single-image prediction
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from PIL import Image
import torch

# load best model
best_model_path = f"{CHECKPOINT_DIR}/best_model.pth"
checkpoint = torch.load(best_model_path, map_location=device)
model.load_state_dict(checkpoint["model_state"])
model.to(device).eval()

def get_preds_probs(model, loader):
    all_labels = []
    all_probs = []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            out = model(imgs)
            probs = torch.sigmoid(out).squeeze().cpu().tolist()
            # ensure list if single item
            if isinstance(probs, float):
                probs = [probs]
            all_probs.extend(probs)
            all_labels.extend(labels.squeeze().tolist())
    return all_labels, all_probs

y_true, y_probs = get_preds_probs(model, test_loader)
y_pred = [1 if p >= 0.5 else 0 for p in y_probs]

accuracy = sum([yt == yp for yt, yp in zip(y_true, y_pred)]) / len(y_true) * 100
print("Test Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["real","fake"]))
print("ROC AUC:", roc_auc_score(y_true, y_probs))

def predict_image(model, image_path, transform):
    img = Image.open(image_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(x)
        p = torch.sigmoid(out).item()
    return p

# 11.**Utilities**

In [ ]:
# Cell 11 — utilities
# Example: download best model to local (if needed)
# from google.colab import files
# files.download(best_model_path)

# Quick test on random batch
imgs, labels = next(iter(val_loader))
imgs, labels = imgs.to(device), labels.to(device)
with torch.no_grad():
    out = model(imgs[:4])
    print("sample outputs:", torch.sigmoid(out).squeeze().cpu().numpy())